In [16]:
import httpx
import pathlib
from decouple import config, AutoConfig
config = AutoConfig(' ')


In [19]:
# pathlib.Path(__file__).resolve() # For python file
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / 'data'
INPUT_DIR = DATA_DIR / 'inputs'
READY_DIR = DATA_DIR / 'ready'
OUTPUT_DIR = DATA_DIR / 'outputs'
GENERATED_DIR = DATA_DIR / 'generated'

In [37]:
API_ACCESS_KEY = config('API_ACCESS_KEY')

headers = {
    "X-API-Key" : API_ACCESS_KEY
}
endpoint = "http://127.0.0.1:8000/predictions"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
# res = httpx.get(endpoint, params={"status":"processing"}, headers=headers)
res = httpx.get(endpoint,
                params={"status":"succeeded"},
                headers=headers,
                )
preds_json = res.json()

In [38]:
preds_json[0]

{'id': '8j6ec6hkk9rme0cpa78bh14vbm',
 'url': '/predictions/8j6ec6hkk9rme0cpa78bh14vbm',
 'status': 'succeeded',
 'created_at': '2025-04-20T06:08:41.370000Z',
 'started_at': '2025-04-20T06:08:41.389645Z',
 'completed_at': '2025-04-20T06:08:59.129586Z'}

In [39]:
BASE_URL = "http://127.0.0.1:8000"
for pred in preds_json:
    path = pred.get('url')
    endpoint = f"{BASE_URL}{path}"
    res = httpx.get(endpoint, headers=headers)
    if res.status_code not in range(200,299):
        continue
    data = res.json()
    files = data.get('files')
    if files is None:
        continue
    print(data)
    obj_id = data.get('id')
    with httpx.Client() as client:
        for i , file_path in enumerate(files):
            fname= pathlib.Path(file_path).name
            outpath = GENERATED_DIR / obj_id / fname
            outpath.parent.mkdir(parents=True, exist_ok=True)
            if outpath.exists():
                continue
            url = f"{BASE_URL}{file_path}"
            print(url)
            res =  client.get(url, headers=headers)
            res.raise_for_status()
            with open(outpath, 'wb') as f:
                f.write(res.content)
    print(data)
    # break



{'id': '8j6ec6hkk9rme0cpa78bh14vbm', 'url': '/predictions/8j6ec6hkk9rme0cpa78bh14vbm', 'model': 'slackroo/superme-sumanth-v1', 'version': '535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49', 'status': 'succeeded', 'created_at': '2025-04-20T06:08:41.370000Z', 'completed_at': '2025-04-20T06:08:59.129586Z', 'files': ['/predictions/8j6ec6hkk9rme0cpa78bh14vbm/files/0.jpg', '/predictions/8j6ec6hkk9rme0cpa78bh14vbm/files/1.jpg'], 'num_outputs': 2}
http://127.0.0.1:8000/predictions/8j6ec6hkk9rme0cpa78bh14vbm/files/0.jpg
http://127.0.0.1:8000/predictions/8j6ec6hkk9rme0cpa78bh14vbm/files/1.jpg
{'id': '8j6ec6hkk9rme0cpa78bh14vbm', 'url': '/predictions/8j6ec6hkk9rme0cpa78bh14vbm', 'model': 'slackroo/superme-sumanth-v1', 'version': '535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49', 'status': 'succeeded', 'created_at': '2025-04-20T06:08:41.370000Z', 'completed_at': '2025-04-20T06:08:59.129586Z', 'files': ['/predictions/8j6ec6hkk9rme0cpa78bh14vbm/files/0.jpg', '/predic

In [40]:
with httpx.Client() as client:
    res = client.get(url, headers=headers)
    print(res)

<Response [200 OK]>
